# Advanced Pandas Techniques for Data Analysis
## Building on Basic Data Cleaning

This notebook covers:
1. Advanced data exploration and visualization
2. Feature engineering techniques
3. Categorical encoding
4. Data aggregation with groupby
5. Advanced filtering and selection
6. Handling outliers
7. Creating derived features

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.precision', 2)

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Load the Titanic dataset
data_path = 'C:\\Users\\vicky\\OneDrive\\Desktop\\Profound\\tested.csv'
df = pd.read_csv(data_path)

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

## 1. Advanced Data Exploration

In [ ]:
# Analyze categorical variables using value_counts()
# This shows the distribution of values in each categorical column

print("=" * 50)
print("PASSENGER CLASS DISTRIBUTION")
print("=" * 50)
print(df['Pclass'].value_counts().sort_index())
print(f"\nPercentage distribution:")
print(df['Pclass'].value_counts(normalize=True).sort_index() * 100)

print("\n" + "=" * 50)
print("GENDER DISTRIBUTION")
print("=" * 50)
print(df['Sex'].value_counts())

print("\n" + "=" * 50)
print("EMBARKATION PORT DISTRIBUTION")
print("=" * 50)
print(df['Embarked'].value_counts())

In [ ]:
# Calculate correlation between numerical features
# Correlation ranges from -1 to 1:
# - Close to 1: strong positive correlation
# - Close to -1: strong negative correlation
# - Close to 0: no correlation

numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
correlation_matrix = df[numerical_cols].corr()

print("Correlation with Survived:")
print(correlation_matrix['Survived'].sort_values(ascending=False))

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1)
plt.title('Correlation Matrix of Numerical Features')
plt.tight_layout()
plt.show()

In [ ]:
# Check unique values and cardinality of each column
# Cardinality = number of unique values
# High cardinality columns (like Name, Ticket) are usually not useful for ML

print("Column Cardinality Analysis:")
print("=" * 60)
for col in df.columns:
    unique_count = df[col].nunique()
    null_count = df[col].isnull().sum()
    print(f"{col:15} | Unique: {unique_count:4} | Nulls: {null_count:3} | Type: {df[col].dtype}")

## 2. Feature Engineering
Creating new features from existing ones can improve model performance

In [ ]:
# Extract title from passenger names
# Titles (Mr., Mrs., Miss., etc.) can be a useful feature

def extract_title(name):
    """Extract title from name string"""
    if pd.isna(name):
        return 'Unknown'
    # Split by comma and period to get the title
    title = name.split(',')[1].split('.')[0].strip()
    return title

# Create new 'Title' column
df['Title'] = df['Name'].apply(extract_title)

print("Title Distribution:")
print(df['Title'].value_counts())

# Group rare titles into 'Other' category
title_counts = df['Title'].value_counts()
rare_titles = title_counts[title_counts < 10].index
df['Title'] = df['Title'].replace(rare_titles, 'Other')

print("\nAfter grouping rare titles:")
print(df['Title'].value_counts())

In [ ]:
# Create Family Size feature
# Family Size = SibSp (siblings/spouses) + Parch (parents/children) + 1 (self)

df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# Create IsAlone feature (1 if traveling alone, 0 otherwise)
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

print("Family Size Distribution:")
print(df['FamilySize'].value_counts().sort_index())

print("\nTraveling Alone:")
print(df['IsAlone'].value_counts())
print(f"\nPercentage traveling alone: {df['IsAlone'].mean() * 100:.1f}%")

In [ ]:
# Create Age Groups by binning continuous Age variable
# This converts numerical age into categorical groups

# First, fill missing ages with median
df['Age'].fillna(df['Age'].median(), inplace=True)

# Create age bins
age_bins = [0, 12, 18, 35, 60, 100]
age_labels = ['Child', 'Teen', 'Young Adult', 'Adult', 'Senior']
df['AgeGroup'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels)

print("Age Group Distribution:")
print(df['AgeGroup'].value_counts().sort_index())

# Visualize age distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
df['Age'].hist(bins=30, edgecolor='black')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution')

plt.subplot(1, 2, 2)
df['AgeGroup'].value_counts().sort_index().plot(kind='bar', color='skyblue', edgecolor='black')
plt.xlabel('Age Group')
plt.ylabel('Count')
plt.title('Age Group Distribution')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create Fare Categories using quantiles
# Quantiles divide data into equal-sized groups

# Fill missing fare
df['Fare'].fillna(df['Fare'].median(), inplace=True)

# Create fare categories based on quartiles
df['FareCategory'] = pd.qcut(df['Fare'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

print("Fare Category Distribution:")
print(df['FareCategory'].value_counts().sort_index())

print("\nFare statistics by category:")
print(df.groupby('FareCategory')['Fare'].describe())

## 3. Data Aggregation with GroupBy
GroupBy allows us to analyze data by categories

In [ ]:
# Analyze survival rates by different features

print("=" * 60)
print("SURVIVAL RATE BY PASSENGER CLASS")
print("=" * 60)
survival_by_class = df.groupby('Pclass')['Survived'].agg([
    ('Count', 'count'),
    ('Survived', 'sum'),
    ('Survival_Rate', 'mean')
])
survival_by_class['Survival_Rate'] = survival_by_class['Survival_Rate'] * 100
print(survival_by_class)

print("\n" + "=" * 60)
print("SURVIVAL RATE BY GENDER")
print("=" * 60)
survival_by_sex = df.groupby('Sex')['Survived'].agg([
    ('Count', 'count'),
    ('Survived', 'sum'),
    ('Survival_Rate', 'mean')
])
survival_by_sex['Survival_Rate'] = survival_by_sex['Survival_Rate'] * 100
print(survival_by_sex)

print("\n" + "=" * 60)
print("SURVIVAL RATE BY AGE GROUP")
print("=" * 60)
survival_by_age = df.groupby('AgeGroup')['Survived'].agg([
    ('Count', 'count'),
    ('Survived', 'sum'),
    ('Survival_Rate', 'mean')
])
survival_by_age['Survival_Rate'] = survival_by_age['Survival_Rate'] * 100
print(survival_by_age)

In [ ]:
# Multi-level grouping: Survival by Class AND Gender

print("Survival Rate by Passenger Class and Gender:")
print("=" * 60)
multi_group = df.groupby(['Pclass', 'Sex'])['Survived'].agg([
    ('Count', 'count'),
    ('Survived', 'sum'),
    ('Survival_Rate', lambda x: f"{x.mean() * 100:.1f}%")
])
print(multi_group)

# Pivot table for better visualization
pivot_survival = df.pivot_table(
    values='Survived',
    index='Pclass',
    columns='Sex',
    aggfunc='mean'
) * 100

print("\nPivot Table - Survival Rate % by Class and Gender:")
print(pivot_survival)

# Visualize
plt.figure(figsize=(10, 6))
pivot_survival.plot(kind='bar', color=['lightcoral', 'skyblue'])
plt.title('Survival Rate by Passenger Class and Gender')
plt.xlabel('Passenger Class')
plt.ylabel('Survival Rate (%)')
plt.legend(title='Gender')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate multiple statistics at once using agg()

print("Comprehensive Statistics by Passenger Class:")
print("=" * 80)
class_stats = df.groupby('Pclass').agg({
    'Age': ['mean', 'median', 'std'],
    'Fare': ['mean', 'median', 'max'],
    'Survived': ['sum', 'mean', 'count'],
    'FamilySize': 'mean'
}).round(2)

print(class_stats)

## 4. Categorical Encoding
Convert categorical variables to numerical for machine learning

In [ ]:
# Method 1: Label Encoding (ordinal encoding)
# Assigns a unique integer to each category
# Best for ordinal data (data with natural order)

from sklearn.preprocessing import LabelEncoder

# Create a copy for encoding experiments
df_encoded = df.copy()

# Encode Sex column
le_sex = LabelEncoder()
df_encoded['Sex_Encoded'] = le_sex.fit_transform(df_encoded['Sex'])

print("Label Encoding for Sex:")
print(df_encoded[['Sex', 'Sex_Encoded']].drop_duplicates().sort_values('Sex_Encoded'))

# Encode Embarked column
df_encoded['Embarked'].fillna('S', inplace=True)  # Fill missing with most common
le_embarked = LabelEncoder()
df_encoded['Embarked_Encoded'] = le_embarked.fit_transform(df_encoded['Embarked'])

print("\nLabel Encoding for Embarked:")
print(df_encoded[['Embarked', 'Embarked_Encoded']].drop_duplicates().sort_values('Embarked_Encoded'))

In [ ]:
# Method 2: One-Hot Encoding (dummy variables)
# Creates binary columns for each category
# Best for nominal data (no natural order)

# One-hot encode Embarked
embarked_dummies = pd.get_dummies(df['Embarked'], prefix='Embarked', drop_first=True)
print("One-Hot Encoding for Embarked:")
print(embarked_dummies.head())

# One-hot encode Sex
sex_dummies = pd.get_dummies(df['Sex'], prefix='Sex', drop_first=True)
print("\nOne-Hot Encoding for Sex:")
print(sex_dummies.head())

# One-hot encode Pclass
pclass_dummies = pd.get_dummies(df['Pclass'], prefix='Pclass', drop_first=False)
print("\nOne-Hot Encoding for Pclass:")
print(pclass_dummies.head())

# Combine all encoded features
df_with_dummies = pd.concat([
    df[['PassengerId', 'Survived', 'Age', 'Fare', 'FamilySize', 'IsAlone']],
    pclass_dummies,
    sex_dummies,
    embarked_dummies
], axis=1)

print("\nDataFrame with One-Hot Encoded Features:")
print(df_with_dummies.head())
print(f"\nShape: {df_with_dummies.shape}")

## 5. Advanced Filtering and Selection

In [ ]:
# Complex filtering with multiple conditions

# Example 1: Young females in first class
young_first_class_females = df[
    (df['Sex'] == 'female') & 
    (df['Pclass'] == 1) & 
    (df['Age'] < 30)
]
print(f"Young females in first class: {len(young_first_class_females)}")
print(f"Survival rate: {young_first_class_females['Survived'].mean() * 100:.1f}%")

# Example 2: Large families (> 4 members) in third class
large_families_3rd = df[
    (df['Pclass'] == 3) & 
    (df['FamilySize'] > 4)
]
print(f"\nLarge families in third class: {len(large_families_3rd)}")
print(f"Survival rate: {large_families_3rd['Survived'].mean() * 100:.1f}%")

# Example 3: Using isin() for multiple values
expensive_tickets = df[df['Fare'] > df['Fare'].quantile(0.75)]
print(f"\nPassengers with expensive tickets (top 25%): {len(expensive_tickets)}")
print(f"Survival rate: {expensive_tickets['Survived'].mean() * 100:.1f}%")

In [ ]:
# Using query() method for cleaner filtering syntax

# Find adult males who survived
result = df.query("Sex == 'male' and Age >= 18 and Survived == 1")
print(f"Adult males who survived: {len(result)}")

# Complex query with variables
min_fare = 50
max_age = 40
result2 = df.query("Fare > @min_fare and Age < @max_age and Pclass in [1, 2]")
print(f"\nPassengers matching complex criteria: {len(result2)}")
print(result2[['Name', 'Age', 'Fare', 'Pclass', 'Survived']].head())

## 6. Outlier Detection and Handling

In [ ]:
# Detect outliers using IQR (Interquartile Range) method
# Outliers are values that fall below Q1 - 1.5*IQR or above Q3 + 1.5*IQR

def detect_outliers_iqr(data, column):
    """Detect outliers using IQR method"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    
    return outliers, lower_bound, upper_bound

# Detect outliers in Fare
fare_outliers, lower, upper = detect_outliers_iqr(df, 'Fare')
print(f"Fare Outliers Detection:")
print(f"Lower bound: ${lower:.2f}")
print(f"Upper bound: ${upper:.2f}")
print(f"Number of outliers: {len(fare_outliers)}")
print(f"\nTop 5 highest fares:")
print(df.nlargest(5, 'Fare')[['Name', 'Fare', 'Pclass', 'Survived']])

# Visualize outliers
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.boxplot(df['Fare'].dropna())
plt.title('Fare Distribution - Box Plot')
plt.ylabel('Fare ($)')

plt.subplot(1, 2, 2)
df['Fare'].hist(bins=50, edgecolor='black')
plt.axvline(upper, color='red', linestyle='--', label=f'Upper bound: ${upper:.2f}')
plt.title('Fare Distribution - Histogram')
plt.xlabel('Fare ($)')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Methods to handle outliers

# Method 1: Cap outliers (Winsorization)
df_capped = df.copy()
upper_cap = df['Fare'].quantile(0.95)
df_capped['Fare_Capped'] = df_capped['Fare'].clip(upper=upper_cap)

print("Original Fare statistics:")
print(df['Fare'].describe())
print("\nCapped Fare statistics:")
print(df_capped['Fare_Capped'].describe())

# Method 2: Log transformation (for skewed data)
df_transformed = df.copy()
df_transformed['Fare_Log'] = np.log1p(df_transformed['Fare'])  # log1p = log(1+x)

# Visualize transformation
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
df['Fare'].hist(bins=50, edgecolor='black')
plt.title('Original Fare Distribution')
plt.xlabel('Fare')

plt.subplot(1, 2, 2)
df_transformed['Fare_Log'].hist(bins=50, edgecolor='black', color='green')
plt.title('Log-Transformed Fare Distribution')
plt.xlabel('Log(Fare + 1)')

plt.tight_layout()
plt.show()

## 7. Final Dataset Preparation for Machine Learning

In [ ]:
# Create a clean, ML-ready dataset

# Start with a fresh copy
df_ml = df.copy()

# Fill missing values
df_ml['Age'].fillna(df_ml['Age'].median(), inplace=True)
df_ml['Fare'].fillna(df_ml['Fare'].median(), inplace=True)
df_ml['Embarked'].fillna('S', inplace=True)

# Select features for modeling
features_to_keep = ['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'IsAlone', 'Embarked', 'Title']
df_ml = df_ml[features_to_keep + ['Survived']]

# Encode categorical variables
from sklearn.preprocessing import LabelEncoder

le_sex = LabelEncoder()
le_embarked = LabelEncoder()
le_title = LabelEncoder()

df_ml['Sex'] = le_sex.fit_transform(df_ml['Sex'])
df_ml['Embarked'] = le_embarked.fit_transform(df_ml['Embarked'])
df_ml['Title'] = le_title.fit_transform(df_ml['Title'])

print("ML-Ready Dataset:")
print(df_ml.head())
print(f"\nShape: {df_ml.shape}")
print(f"\nData types:")
print(df_ml.dtypes)
print(f"\nMissing values:")
print(df_ml.isnull().sum())

In [ ]:
# Split into features and target
X = df_ml.drop('Survived', axis=1)
y = df_ml['Survived']

print("Features (X):")
print(X.head())
print(f"\nShape: {X.shape}")

print("\nTarget (y):")
print(y.head())
print(f"Shape: {y.shape}")

# Feature scaling (optional but recommended for many ML algorithms)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print("\nScaled Features:")
print(X_scaled_df.head())
print("\nFeature Statistics After Scaling (should have mean≈0, std≈1):")
print(X_scaled_df.describe().round(2))

## Summary

### What We Learned:

1. **Advanced Exploration**
   - Value counts and distribution analysis
   - Correlation analysis
   - Cardinality checking

2. **Feature Engineering**
   - Extracting titles from names
   - Creating family size features
   - Binning continuous variables
   - Quantile-based categorization

3. **GroupBy Operations**
   - Single and multi-level grouping
   - Custom aggregations
   - Pivot tables

4. **Categorical Encoding**
   - Label encoding for ordinal data
   - One-hot encoding for nominal data

5. **Advanced Filtering**
   - Complex boolean conditions
   - Query method

6. **Outlier Handling**
   - IQR method detection
   - Capping (Winsorization)
   - Log transformation

7. **ML Preparation**
   - Feature selection
   - Encoding
   - Scaling

### Next Steps:
- Train machine learning models
- Cross-validation
- Hyperparameter tuning
- Model evaluation